In [1]:
# LT 30/08/2022
# gnomAD v4 prototyping test using gnomAD v4 exomes test vds

import hail as hl
hl.init(default_reference='GRCh38',
        spark_conf = {
            'spark.hadoop.fs.gs.requester.pays.mode': 'CUSTOM',
            'spark.hadoop.fs.gs.requester.pays.buckets': 'gnomad',
            'spark.hadoop.fs.gs.requester.pays.project.id': 'constraint-232598',
        })


2022-09-01 00:52:44 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://5c0cee79c122:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.97-937922d7f46c
LOGGING: writing to /home/jupyter/hail-20220901-0052-0.2.97-937922d7f46c.log


In [2]:
# gnomAD v4 exomes test VDS
small_vds = hl.vds.read_vds('gs://gnomad/v4.0/raw/exomes/testing/gnomad_v4.0_test.vds')

# densify as a MatrixTable
mt = hl.vds.to_dense_mt(small_vds)

# remove REF/REF rows left after densification
mt = mt.filter_rows(hl.len(mt.alleles) < 2, keep=False)

# get alleles frequencies
mt = mt.transmute_entries(GT=hl.experimental.lgt_to_gt(mt.LGT, mt.LA))
mt = hl.variant_qc(mt)

# keep only the rows
# persist to avoid double evaluation later 
ht = mt.rows().checkpoint('gnomad_v4.0_test.ht', overwrite=True)

2022-08-30 00:06:38 Hail: INFO: wrote table with 3268516 rows in 3600 partitions to gnomad_v4.0_test.ht


In [3]:
#hl.stop()

In [4]:
ht.count()

3268516

In [2]:
# try naive vep without install
# cannot perform local install of vep

In [3]:
# read small table and repartion to check performance
ht = hl.read_table('gnomad_v4.0_test.ht')

In [4]:
ht.n_partitions()

3600

In [6]:
ht_repart = ht.repartition(1000, shuffle=False).checkpoint('gnomad_v4.0_test_repart.ht')

2022-08-31 06:05:45 Hail: WARN: coalesced to 996 partitions, less than requested 1000
2022-08-31 06:06:11 Hail: INFO: wrote table with 3268516 rows in 996 partitions to gnomad_v4.0_test_repart.ht
    Total size: 270.91 MiB
    * Rows: 270.91 MiB
    * Globals: 11.00 B
    * Smallest partition: 541 rows (47.49 KiB)
    * Largest partition:  8925 rows (829.98 KiB)


In [8]:
ht_repart = ht.repartition(1000, shuffle=True).checkpoint('gnomad_v4.0_test_repart.ht', overwrite=True)

2022-08-31 06:38:26 Hail: INFO: wrote table with 3268516 rows in 1000 partitions to gnomad_v4.0_test_repart.ht
    Total size: 270.92 MiB
    * Rows: 270.92 MiB
    * Globals: 11.00 B
    * Smallest partition: 1312 rows (110.64 KiB)
    * Largest partition:  6073 rows (506.51 KiB)


In [2]:
# gnomAD v4 exomes test VDS
vds = hl.vds.read_vds('gs://gnomad/v4.0/raw/exomes/testing/gnomad_v4.0_test.vds')
vds.variant_data.count()

(3268516, 649)